# **Projet de programmation mathématique avancée**
## **- Kidney exchange problem -**
### Anna Kerebel, Hiba Shaimed, Kefan Sun, Victor Klötzer, Vinh Nguyen

 *14 mai 2021*

---

### Sommaire
1. Importation des packages Julia et des fonctions
2. Première version du Branch & Price  
    a. Formulation mathématique  
    b. Règle de branchement  
    c. Colonne artificielle  
    d. Stockage des colonnes   
    e. Test sur l'instance 071 et première comparaison avec le MIP
3. Améliorations  
    a. Utilisation de variables globales pour les différentes versions  
    b. Changement de règles de branchement  
    c. Ajout d'une heuristique pour démarrer l'algo  
    
    
$~$

Dans ce projet, nous nous sommes intéressé au problème de Kidney exchanges en ne traitant que le cas sans la présence de donneurs altruistes. Nous nous sommes par contre penchés sur différentes partie de l'algorthme de Branch and Price pour essayer de l'améliorer, i.e. le rendre plus efficace. Nous présentons donc ici notre première implémentation dans un premier temps, puis nous expliquerons les améliorations que nous avons essayé de faire.

# 1. Importation des packages Julia et des fonctions

In [1]:
# Pakages utilisés
using JuMP 
using Gurobi
using DelimitedFiles
using LightGraphs
using MetaGraphs
using NBInclude
const GUROBI_ENV = Gurobi.Env()
using Plots
using Statistics
const ϵ = 0.00001

Academic license - for non-commercial use only - expires 2021-06-26


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


Your GR installation is incomplete. Rerunning build step for GR package.


ERROR: LoadError: LoadError: InitError: Evaluation into the closed module `GR` breaks incremental compilation because the side effects will not be permanent. This is likely due to some other module mutating `GR` with `eval` during precompilation - don't do this.
Stacktrace:
 [1] eval at .\boot.jl:331 [inlined]
 [2] __init__() at C:\Users\Victor\.julia\packages\GR\Hsil0\src\GR.jl:335
 [3] _include_from_serialized(::String, ::Array{Any,1}) at .\loading.jl:697
 [4] _require_search_from_serialized(::Base.PkgId, ::String) at .\loading.jl:782
 [5] _require(::Base.PkgId) at .\loading.jl:1007
 [6] require(::Base.PkgId) at .\loading.jl:928
 [7] require(::Module, ::Symbol) at .\loading.jl:923
 [8] include(::Function, ::Module, ::String) at .\Base.jl:380
 [9] include at .\Base.jl:368 [inlined]
 [10] include(::String) at C:\Users\Victor\.julia\packages\Plots\vVVub\src\Plots.jl:1
 [11] top-level scope at C:\Users\Victor\.julia\packages\Plots\vVVub\src\Plots.jl:218
 [12] include(::Function, ::Module

LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to C:\Users\Victor\.julia\compiled\v1.5\Plots\ld3vC_xtJpQ.ji.

In [ ]:
using Pkg

In [2]:
# Importation des fonctions

@nbinclude("01_data.ipynb")
@nbinclude("02_node.ipynb")
@nbinclude("03_master.ipynb")
@nbinclude("04_subproblem.ipynb")
@nbinclude("05_branch_and_price.ipynb")
@nbinclude("06_MIP.ipynb")
@nbinclude("07_comparaison.ipynb")

compare (generic function with 1 method)

Le premier fichier "01_data.ipynb" contient la fonction permettant d'importer des instances d'essai du problème de Kidney exchange.  
Les fichiers numérotés de 02 à 05 contiennent toutes les fonctions permettant d'implémenter notre algorithme de Branch and Price sur ce problème. Différentes versions de fonctions ont été codées pour certaines partie de cet algorithme et seront détaillées par la suite.   
Le fichier "06_MIP.ipynb" contient une fontion résolvant directement le problème MIP tel qu'écrit dans le sujet du projet.  
Enfin, le fichier "07_comparaison.ipynb" nous permet finalement de comparer entre elles les différentes versions de notre Branch and Price, ainsi que de les comparer au l'implémentation MIP fait dans le fichier 06.  

Vous trouverez dans chacun de ces fichier, des descriptions et des commentaires plus détaillés.

Ci-dessous, on importe justement les données de l'instance 071 dont nous nous servirons comme instance jouet pour montrer et dérouler nos codes dans la suite de ce rapport.

In [4]:
# Importation des données

data_folder = string(join(split(@__DIR__, '\\')[1:end-1],'\\'),"\\data") # Afin d'accéder au dossier 'data'
instance = "MD-00001-00000071"
wmd_file = joinpath(data_folder, join([instance, ".wmd"]))
dat_file = joinpath(data_folder, join([instance, ".dat"]))
global GRAPH = read_graph(wmd_file, dat_file)
global VERTICES = 1:nv(GRAPH)
global EDGES = [(e.src,e.dst) for e in edges(GRAPH)]
global L = 3
global WEIGHTS = Dict{Tuple{Int,Int},Float32}((e.src,e.dst) => get_prop(GRAPH, e, :weight) for e in edges(GRAPH))

# Affichage d'informations sur les données
println("Nombre de couples donneur/receveur             : $(nv(GRAPH))")
println("Nombre de transferts possibles (nombre d'arcs) : $(ne(GRAPH))")
println("Longueur maximale autorisée pour les cycles    : $L")

*********** Read instance MD-00001-00000071 ***********
Nombre de couples donneur/receveur             : 64
Nombre de transferts possibles (nombre d'arcs) : 1191
Longueur maximale autorisée pour les cycles    : 3


# 2. Première version du Branch & Price  
## a. Formulation mathématique

Pour pouvoir implémenter un Branch and Price, il nous fallait avant toute chose écrire le problème maître restreint, le sous-problème ainsi que retrouver une formulation de la relaxation lagrangienne du problème entier de départ afin de pouvoir calculer des bornes duales dans l'algorithme de BP.  
  
Pour ce faire, nous avons décidé d'introduire deux familles de variables $x$ et $y$, respectivement pour les arcs et les sommets. Nous obtenons ainsi les reformulations suivantes : 
  
Problème maître restreint ($\mathcal C_L^R \subset  \mathcal C_L$) :
$$\begin{array}{rlccrlc}
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c w_c  & & \iff &
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c \sum_{(i,j) \in E}\bar x_{(i,j)}^c w_{(i,j)} \\
\text{sous} & \displaystyle\sum\limits_{c\in \mathcal C_L^R \,|\, i\in c} z_c \le 1 & \forall i\in V & &
\text{sous} & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c \,\bar y_i^c \le 1 & \forall i\in V ~~~\big\vert~~(\pi_i)_{i\in V}\\
            & z \in [0,1]^{|\mathcal C_L^R|} & & & & z \in [0,1]^{|\mathcal C_L^R|}
\end{array}$$

où :
*  $\bar x_{(i,j)}^c = 1$ si l'arc $(i,j)$ se trouve dans le cycle $c$, et $\bar x_{(i,j)}^c = 0$ sinon
*  $\bar y_i^c = 1$ si le sommet $i$ fait partie du cycle $c$, et $\bar y_i^c = 0$ sinon

Dans ce Branch and Price, les colonnes représenteront ainsi des cycles de couples donneur/receveur. L'ensemble des contraintes "simples" énumère donc l'ensemble des cycles de longueur au plus $L$ qui peuvent être trouvés dans une population de donneurs/receveurs.

$~$  

Pour le sous-problème, nous avons ainsi essayé de créer une formulation PLNE, et donc d'imposer des contraintes permettant de créer ces cycles de longueurs au plus $L$. Voici la formulation que nous avons utilisée :

Sous-problème :
$$\begin{array}{rlcrlc}
\max        & \displaystyle\sum\limits_{(i,j)\in E} x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i^c & \iff &
\max        & \displaystyle\sum\limits_{(i,j)\in E} x_{(i,j)} w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i \\
\text{sous} & c\in\mathcal C_L & &
\text{sous} & \displaystyle\sum\limits_{(i,j) \in\delta^-(i)} x_{(i,j)} = y_i & \forall i\in V\\
      & & & & \displaystyle\sum\limits_{(j,i) \in\delta^+(i)} x_{(j,i)} = y_i & \forall i\in V\\
      & & & & \displaystyle\sum\limits_{(i,j) \in E} x_{(i,j)} \le L \\
      & & & & \displaystyle\sum\limits_{(i,j) \in E} x_{(i,j)} \ge 2 \\
      & & & & x \in \{0,1\}^{|E|}, y \in \{0,1\}^{|V|}
\end{array}$$

Les deux premières contraintes permettent d'assurer la création d'un cycle (ou de plusieurs cycles) : en un sommet ne rentre au plus qu'un seul arc et si un arc est entré alors il en ressort aussi un.   
La troisème contrainte impose au cycle d'être au plus de longueur $L$.  
La quatrième contrainte permet d'interdire que le cycle vide soit utilisé, car dans ce problème un cycle contiendra au minimum deux arcs.  
  
Il faut néanmoins bien noter que dès lors que $L>3$, ces quatres contraintes n'imposent pas la connexité du graphe trouvé. Par exemple pour $L=4$, on peut ainsi tout à fait trouver deux cycles séparés de taille $2$ chacun. Un problème non négligable de cette particularité, est que plus $L$ sera grand par rapport à $4$ et plus il y aura des répétitions dans l'énumération de petits cycles. Pour reprendre l'exemple de tout à l'heure, les deux cycles de tailles $2$ sont présents une fois ensemble dans une colonne, et seront aussi présents de manière séparée dans deux autres colonnes. Cela ajoute ainsi de la symmétrie, ce qui n'est pas optimal, et impliquerait donc notamment des temps de calcul beaucoup plus long.  
  
Dans ce projet on se limitera ainsi à $L\le 3$, car dans ce cas la formulation proposée ci-dessus permet bien de créer qu'un seul cycle de taille inférieure ou égale à $3$. Bien que cela ne soit pas exaustif, limiter $L$ à 3 au plus n'est pas non plus complètement aberrant. En effet pour des raisons médicales (de personnels et techniques) et afin de garantir le bon déroulé de ces échanges de reins, les cycles ne doivent pas être très grands. Dans le papier "[1] Clearing Algorithms for Barter Exchange Markets Enabling Nationwide Kidney Exchanges" les auteurs se limitaient ainsi également pour des raisons de temps de calcul à $L\le 3$  et évoquaient même qu'autoriser des cycles de longueurs plus grandes que $3$ n'améliore souvent pas vraiment la solution du problème (voir par exemple paragraphe 2 de la section 1.1, ou paragraphe 2 de la section 5.2.3).

$~$  

Voici enfin une reformulation de la relxation lagrangienne du problème entier écrit directement à partir de la reformulation de Dantzig-Wolfe donnée dans le sujet :

Relaxation lagrangienne (avec $\lambda_i \ge 0$, $\forall i\in V$) :
$$\begin{array}{rlcrl}
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)}\big) + \sum\limits_{i\in V}\lambda_i \big(1 - \sum\limits_{c\in \mathcal C_L} z_c \, \bar y_i^c \big)\\
\text{sous} & z \in [0,1]^{|\mathcal C_L|}
\end{array}$$
où on peut majorer la solution optimale comme suit :
$$\begin{aligned}\displaystyle\underset{z}{\max}~\sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)}\big) + \sum\limits_{i\in V}\lambda_i \big(1 - \sum\limits_{c\in \mathcal C_L} z_c \, \bar y_i^c \big) &= \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + \sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \bar y_i^c \big)\\
~~~~~ &\le \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + {\bf u}_\text{SP}^*\sum\limits_{c\in \mathcal C_L} z_c \\
~~~~~ &\le \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + {\bf u}_\text{SP}^*\times M
\end{aligned}$$


avec $\displaystyle{\bf u}_\text{SP}^* := \underset{c\in\mathcal C_L}{\max}~\sum\limits_{(i,j)\in E} x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i^c~~$ (solution optimale du sous-problème)

et $M$ est un majorant de $\displaystyle\sum\limits_{c\in \mathcal C_L} z_c$, i.e. le nombre de cycles trouvés finalement à la résolution du problème.  

Nous avons ainsi pris comme borne duale du problème entier (borne supérieure ici) la valeur suivante :
$$\sum\limits_{i\in V}\pi_i^* + M\times {\bf u}_\text{SP}^*$$
avec $M=\frac{|V|}2$ car dans le cas où tous les sommets seraient utilisés dans la solution, on pourrait au plus faire $\frac{|V|}2$ cycles de taille $2$.
Cette borne duale est calculée et mise à jour dans les algortihmes de générations de colonnes exécutés en chaque nœud.

## b. Règle de branchement

Dans la première version de notre Branch and Price, nous avons décidé de brancher assez classiquement sur les arcs, i.e. sur les $x_{(i,j)}$ fractionnaires qui sont obtenus à la résolution d'un nœud de l'arbre de branchement. Nous branchons ainsi sur le premier $x_{(i,j)}$ fractionnaire trouvé (si la solution n'était pas entière).
  
Concernant la manière avec laquelle les contraintes sont transmises aux nœuds enfants, nous avons utilisé l'option 2 (comme dans le cours), à savoir que pour le problème maître restreint l'ensemble des colonnes sur lequels il peut être résolu est diminué, et que l'on ajoute les contraintes de branchement dans les sous-problèmes.  

Nous avons aussi décidé que les nœuds enfants hériteraient des colonnes générées par leur parent qui vérifient les contraintes de branchement afin de démarrer toujours avec un ensemble de colonnes intéressantes déjà grand. Ainsi peu de nouvelles colonnes doivent être ajoutées dans les nœeuds autre que le nœud racine.

Cette première règle de branchement est utilisé en passant la valeur `"sur_arcs"` à la variable globale `BRANCHING_VERSION` comme vous pourrez le voir ci-dessous.

## c. Colonne artificielle

Afin de pouvoir démarrer la génération de colonne dans le nœud racine, nous avons dans cette première version décidé de commencer avec une colonne artificielle. Cette colonne artificielle considère que tous les sommets sont impliqués dans un cycle mais que ce cycle a pour poids la valeur minimale du poids attribué à un arc (qui doit tout de même être $>0$). Ainsi, cette colonne artificielle doit être remplacée dès qu'une "vraie" colonne est trouvée et si jamais elle reste dans la solution finale, c'est que le problème n'était pas réalisable.

Dans les nœuds enfants, afin de s'assurer qu'au moins une colonne est héritée du nœud parent pour démarrer là aussi la génération de colonnes, on ajoute également toujours cette colonne artificielle.

## d. Stockage des colonnes

Dans notre problème, une colonne $c$ devrait être uniquement composée de son poids $w_c$ ainsi que des variables $y_i$ pour $i\in V$ indiquant quel sommet est inclus dans ce cycle. Seulement afin de pouvoir brancher sur les arcs, il nous fallait également conserver les arcs, i.e. les variables $x_{(i,j)}$. Afin de limiter le stockage, nous avons donc seulement conservé les arcs utilisés dans le cycle (i.e les $x_{(i,j)}=1$) qui sont très peu nombreux. Une "colonne" `column` est donc représenté dans notre code par un dictionnaire avec trois attributs :
* `column["cost"]` contenant le poids du cycle associé
* `column["vertices"]` contenant l'ensemble des valeurs prise par les variables $y_i, \forall i\in V$
* `column["oneedges"]` contenant sous forme de tuples `(i,j)` les arcs consituant ce cycle.

## e. Test sur l'instance 071 et première comparaison avec le MIP

On teste cette première implémentation ci-dessous sur notre exemple jouet l'instance 071. On reviendra dans la prochaine section sur les différentes versions de codes que l'on peut choisir à l'aide des trois variables `BP_INIT_VERSION`, `BRANCHING_VERSION` et `PROCESSING_RULE`. La variables `SHOW_ADDED_CYCLES` permet d'afficher ou non les cycles ajoutés pendant la génération de colonnes.

In [5]:
global BP_INIT_VERSION   = "artificielle" 
global BRANCHING_VERSION = "sur_arcs"  
global PROCESSING_RULE   = "profondeur" 
global SHOW_ADDED_CYCLES = false                # affichage ou non des cycles
result = @timed solve_BP()

Liste des noeuds qu'il reste à traiter : [1]
NOEUD n°1 :


LoadError: UndefVarError: ϵ not defined

In [13]:
result_MIP = @timed solve_MIP()

Academic license - for non-commercial use only - expires 2021-06-26
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 64 rows, 1595 columns and 4644 nonzeros
Model fingerprint: 0xfb52e18e
Variable types: 0 continuous, 1595 integer (1595 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 39.0000000
Presolve removed 5 rows and 413 columns
Presolve time: 0.02s
Presolved: 59 rows, 1182 columns, 3415 nonzeros
Found heuristic solution: objective 41.0000000
Variable types: 0 continuous, 1182 integer (1182 binary)

Root relaxation: objective 4.700000e+01, 192 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0   

(value = (47.0, [-0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0  …  0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), time = 0.5115986, bytes = 30225843, gctime = 0.0254618, gcstats = Base.GC_Diff(30225843, 13, 0, 520792, 228, 0, 25461800, 1, 0))

Comme on peut le voir ci-dessus cette première implémentation du Branch and Price semble fonctionner et retourne d'ailleurs la même valeur que la formulation MIP. Néanmoins on peut constater que le MIP est beaucoup plus rapide sur cette instance qui n'est déjà pas très grande (et l'écart est encore plus grand avec des instances avec 128 sommets par exemple). On remarque notamment que l'arbre de branchement est assez profond et assez déséquilibré dans le sens où c'est presque toujours le nœud pour laquelle la contrainte $x_{(i,j)}=1$ qui est élagé. Ceci est tout à fait logique puisqu'imposer la présence d'un arc est une contrainte extrêmement forte et donc très très souvent pas intéressant. On se retrouve ainsi avec une structure d'arbre de la forme : 
```
/\
 /\
  /\
   /\
    ...etc.
```


Nous avons essayé d'améliorer ce branchement dans un second temps.

# 3. Améliorations

Dans cette partie, nous avons essayé de mieux équilibrer les branchements de l'arbre, afin qu'il soit moins profond. Et à défaut de le rendre plus équilibré, nous avons aussi essayé de le rendre plus rapide en temps et nombre de nœuds à traiter.

Nous avons ainsi ajouter deux possibilité de parcours de l'arbre : il peut désormais être parcouru en profondeur comme précédemment, ou bien en largeur ou alors avec le critière du "meilleur d'abord", à savoir que l'on commence toujours par le nœud actif avec la meilleure borne duale (i.e. la plus petite ici). Ce paramètre peut être choisi à l'aide de la variable `PROCESSING_RULE`.  

Afin d'améliorer ensuite le démarrage de l'algorthime, son initialisation, nous avons implémenter une heuristique qui permet de commencer la génération de colonne au nœud racine avec un ensemble de colonnes déjà intéressant. Cette heuristique que vous retrouver dans la fonction `KEP_heuristic` dans le fichier "05_branch_and_pric.ipynb" recherche tout simplement petit à petit des cycles de longueurs au plus $L$ à l'aide d'un parcours en profondeur. La recherche commence par un sommet et dès qu'un cycle est trouvé, on continue en commmençant par un autre sommet et en cherchant des cycles qui ne comportent pas de sommets déjà utilisés dans un cycle trouvé précédemment. L'utilisation ou non de cette heuristique pour initialiser le Branch and Price peut être choisie à l'aide de la variable `BRANCHING_VERSION`.

In [10]:
global BP_INIT_VERSION   = "artificielle"       # Choisir "artificielle" ou "heuristique"
global BRANCHING_VERSION = "sur_arcs"           # Choisir "sur_arcs" ou "sur_sommets"
global PROCESSING_RULE   = "profondeur"         # Choisir "profondeur", "largeur" ou "meilleur_d_abord"
global SHOW_ADDED_CYCLES = false                # affichage ou non des cycles
result = @timed solve_BP()

Liste des noeuds qu'il reste à traiter : [1]
NOEUD n°1 :


LoadError: UndefVarError: ϵ not defined

In [4]:
list_cycles = KEP_heuristic()
primal_bound = 0
for c in list_cycles
    primal_bound += length(c)-1
end
primal_bound

57

In [4]:
global BP_INIT_VERSION   = "heuristique"        # Choisir "artificielle" ou "heuristique"
global BRANCHING_VERSION = "sur_sommets"        # Choisir "sur_arcs" ou "sur_sommets"
global PROCESSING_RULE   = "profondeur"         # Choisir "profondeur", "largeur" ou "meilleur_d_abord"
global SHOW_ADDED_CYCLES = false 

res = temps("MD-00001-00000112")

********* Read instance MD-00001-00000112 *********
Academic license - for non-commercial use only - expires 2021-06-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 128 rows, 8288 columns and 24437 nonzeros
Model fingerprint: 0xd42d4df6
Variable types: 0 continuous, 8288 integer (8288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 68.0000000
Presolve removed 3 rows and 1302 columns
Presolve time: 0.04s
Presolved: 125 rows, 6986 columns, 20566 nonzeros
Found heuristic solution: objective 70.0000000
Variable types: 0 continuous, 6986 integer (6986 binary)

Root relaxation: objective 8.300000e+01, 477 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_6 == y_8 et y_6 != y_8
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20, 21]
NOEUD n°21 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20]
NOEUD n°20 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_6 == y_27 et y_6 != y_27
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [22, 23]
NOEUD n°23 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne infér

(128, 0.1940956, 42.828152699, 41)

In [5]:
result_MIP = @timed solve_MIP()

Academic license - for non-commercial use only - expires 2021-06-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 128 rows, 6870 columns and 20195 nonzeros
Model fingerprint: 0xb3c3783d
Variable types: 0 continuous, 6870 integer (6870 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 59.0000000
Presolve removed 11 rows and 1215 columns
Presolve time: 0.06s
Presolved: 117 rows, 5655 columns, 16615 nonzeros
Variable types: 0 continuous, 5655 integer (5655 binary)

Root relaxation: objective 7.800000e+01, 342 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      78.0000000   78.

(value = (78.0, [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]), time = 15.350975801, bytes = 2095757147, gctime = 1.014769701, gcstats = Base.GC_Diff(2095757147, 30, 0, 38939315, 5977, 0, 1014769701, 24, 1))

In [31]:
# Contient dans l'ordre BP_INIT_VERSION, BRANCHING_VERSION puis PROCESSING_RULE
liste_params = [["heuristique","sur_sommets","profondeur"], ["artificielle","sur_sommets","profondeur"]]
liste_instances = ["MD-00001-00000112","MD-00001-00000113","MD-00001-00000001","MD-00001-00000002"]
compare(liste_instances, liste_params)

********* Read instance MD-00001-00000112 *********
Academic license - for non-commercial use only - expires 2021-06-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 128 rows, 8288 columns and 24437 nonzeros
Model fingerprint: 0xd42d4df6
Variable types: 0 continuous, 8288 integer (8288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 68.0000000
Presolve removed 3 rows and 1302 columns
Presolve time: 0.08s
Presolved: 125 rows, 6986 columns, 20566 nonzeros
Found heuristic solution: objective 70.0000000
Variable types: 0 continuous, 6986 integer (6986 binary)

Root relaxation: objective 8.300000e+01, 477 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

NOEUD n°18 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_6 == y_8 et y_6 != y_8
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20, 21]
NOEUD n°21 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20]
NOEUD n°20 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_6 == y_27 et y_6 != y_27
LB=74.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [22, 23]
NOEUD n°23 :
 |  Relaxation du noeud résolue à l'optimalité avec l

 |  Meilleure solution réalisable (entière) de valeur 3.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 6.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 9.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 12.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 15.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 18.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 21.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 24.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 27.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 66.0 trouvée 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_5 == y_91 et y_5 != y_91
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [2, 3]
NOEUD n°3 :
 |  Rel

NOEUD n°23 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [22]
NOEUD n°22 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 82.99999999999997
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_12 == y_119 et y_12 != y_119
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [24, 25]
NOEUD n°25 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [24]
NOEUD n°24 :
 |  Relaxation du noeud résolue à l'optima

 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_53 == y_57 et y_53 != y_57
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [46, 47]
NOEUD n°47 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [46]
NOEUD n°46 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur y_44 == y_84 et y_44 != y_84
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [48, 49]
NOEUD n°49 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure :

NOEUD n°10 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 77.99999999999999
 |                                               et la borne supérieure : 78.0
Deux noeuds sont créés en branchant sur y_2 == y_72 et y_2 != y_72
LB=69.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [12, 13]
NOEUD n°13 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=69.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [12]
NOEUD n°12 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 78.0
 |                                               et la borne supérieure : 78.0
Deux noeuds sont créés en branchant sur y_4 == y_33 et y_4 != y_33
LB=69.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [14, 15]
NOEUD n°15 :
 |  Relaxation du noeud résolue à l'optimalité avec

 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 78.0
 |                                               et la borne supérieure : 78.0
Deux noeuds sont créés en branchant sur y_3 == y_51 et y_3 != y_51
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [4, 5]
NOEUD n°5 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [4]
NOEUD n°4 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 78.0
 |                                               et la borne supérieure : 78.0
Deux noeuds sont créés en branchant sur y_1 == y_20 et y_1 != y_20
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [6, 7]
NOEUD n°7 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |     

 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [26]
NOEUD n°26 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 78.00000000000001
 |                                               et la borne supérieure : 78.0
Deux noeuds sont créés en branchant sur y_28 == y_104 et y_28 != y_104
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [28, 29]
NOEUD n°29 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=27.0, UB=78.0 
Liste des noeuds qu'il reste à traiter : [28]
NOEUD n°28 :
 |  Relaxation du noeud résolue à l'optimalité avec la 

LoadError: MethodError: no method matching getindex(::Set{Float32}, ::Int64)

In [ ]:
global SHOW_ADDED_CYCLES = false 
# Contient dans l'ordre BP_INIT_VERSION, BRANCHING_VERSION puis PROCESSING_RULE
liste_params1 = [["heuristique","sur_sommets","profondeur"], ["artificielle","sur_sommets","profondeur"],

                ["heuristique","sur_sommets","largeur"], ["artificielle","sur_sommets","largeur"],
    
            ["heuristique","sur_sommets","meilleur_d_abord"], ["artificielle","sur_sommets","meilleur_d_abord"]]

liste_instances = ["MD-00001-00000003","MD-00001-00000006","MD-00001-00000009","MD-00001-00000033","MD-00001-00000036",
                   "MD-00001-00000039","MD-00001-00000073","MD-00001-00000076","MD-00001-00000079","MD-00001-00000113",
                   "MD-00001-00000116","MD-00001-00000119"]
tailles_instances, mean_tps_MIP1, mean_tps_BP_par_param1, mean_nb_noeuds_par_param1 = compare(liste_instances, liste_params1)

# Conclusion

Amélioration possible sur le plus fractionnaire

Pas au delà de L=3